In [1]:
# Importa as bibliotecas necessárias
import pandas as pd
import numpy as np

In [2]:
# Configura o estilo de exibição de floats
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
# Carrega e lê o dataframe
sales = pd.read_csv('C:/Users/mathe/Projetos Dados/EcommerceSalesAnalysis/App/Data/Online Retail.csv', delimiter=';', encoding='latin1', decimal=',')
sales.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.00,United Kingdom


#### Deve ser adicionada uma coluna de Preço Final de venda/devolução para cada transação

In [4]:
# Checa as dimensões do dataframe
sales.shape

(541909, 8)

#### Uma análise do Dataset mostrou que além de registrar as operações de venda e cancelamento dos produtos, esse dataset também registra operações manuais de ajustes, descontos ofertados, taxas, comissões e outras operações.

In [5]:
# Filtra as operações que não se referem a produtos
def not_number(value):
    return not value[0].isdigit()

# Filtra todos os registros que não vendas/cancelamento de produtos
not_product_lines = sales[sales['StockCode'].apply(not_number)]

# Obtém os valores únicos das linhas
not_product = not_product_lines['StockCode'].unique()

# Imprime os valores únicos que não começam com números
print(not_product)

['POST' 'D' 'C2' 'DOT' 'M' 'BANK CHARGES' 'S' 'AMAZONFEE' 'DCGS0076'
 'DCGS0003' 'gift_0001_40' 'DCGS0070' 'm' 'gift_0001_50' 'gift_0001_30'
 'gift_0001_20' 'DCGS0055' 'DCGS0072' 'DCGS0074' 'DCGS0069' 'DCGS0057'
 'DCGSSBOY' 'DCGSSGIRL' 'gift_0001_10' 'PADS' 'DCGS0004' 'DCGS0073'
 'DCGS0071' 'DCGS0068' 'DCGS0067' 'DCGS0066P' 'B' 'CRUK']


#### Portanto, como a análise está focada apenas na VENDA e CANCELAMENTO dos PRODUTOS, os registros citados devem ser excluídos no pré-processamento.

#### Essas operações são identificadas pela coluna *[StockCode]*, em que seus valores iniciam-se por uma letra. As linhas que contem os dados presentes em "not_product", conforme mostrado acima, devem ser excluídas, por não se tratarem de vendas/cancelamentos de produtos.

In [6]:
# Checa os tipos dos dados
sales.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

#### Pela análise dos tipos dos dados desse dataset, devem ser aplicadas transformações nos tipos de dados durante o pré-processamento.

In [7]:
# Resumo dos dados
sales[['Quantity', 'UnitPrice']].describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.00,9.55,218.08,-80995.00,1.00,3.00,10.00,80995.00
UnitPrice,541909.00,4.61,96.76,-11062.06,1.25,2.08,4.13,38970.00


#### A análise das estatísticas descritivas dos dados mostra o valor mínimo de *[UnitPrice]* sendo negativo. Portanto, devem ser excluídos no pré-processamento os registros com valor unitário menor ou igual a zero, pois não configuram-se como vendas ou devoluções. 
#### Provavelmente são decorrentes de erros de registro

In [8]:
# Verifica a presença de registros duplicados no dataset
cols_list = sales.columns
duplicates = sales.duplicated(subset=cols_list, keep=False)
sales[duplicates].sort_values('Description').head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
483390,577503,72800B,4 PURPLE FLOCK DINNER CANDLES,1,20/11/2011 12:34,2.55,18110.00,United Kingdom
483414,577503,72800B,4 PURPLE FLOCK DINNER CANDLES,1,20/11/2011 12:34,2.55,18110.00,United Kingdom
160498,550459,72800B,4 PURPLE FLOCK DINNER CANDLES,1,18/04/2011 13:17,2.55,18116.00,United Kingdom
160509,550459,72800B,4 PURPLE FLOCK DINNER CANDLES,1,18/04/2011 13:17,2.55,18116.00,United Kingdom
460074,575895,23345,DOLLY GIRL BEAKER,1,11/11/2011 14:50,1.25,17052.00,United Kingdom


#### Como foi notada a presença de duplicatas no dataset, será necessário removê-las durante o pré-processamento

In [9]:
# Checa valores ausentes
sales.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

#### Há presença de valores ausentes na coluna *[CustomerID].* e na coluna *[Description]*

#### Esses valores nulos devem ser tratados de acordo com a análise realizada, principalmente para a coluna *[CustomerID]*, que possui uma grande quantidade de valores nulos.

#### Como essa quantidade é significativa, as linhas com valores nulos de *[CustomerID]* só serão removidas durante a análise dos clientes, mas serão mantidas nas análises de produtos, países e sazonalidade, já que a remoção iria afetar profundamente essas análises.

In [10]:
# Verifica a existência de nomes de produtos iguais para diferentes StockCodes
duplicates_descriptions = sales.groupby('Description')['StockCode'].nunique()

# Filtra os casos onde há mais de um StockCode para a mesma Description
duplicates_descriptions = duplicates_descriptions[duplicates_descriptions > 1]

# Imprime as Description com mais de um StockCode associado
print("Descriptions com mais de um StockCode associado:")
print(duplicates_descriptions.sort_values(ascending=False))

Descriptions com mais de um StockCode associado:
Description
check                                  146
?                                       47
damaged                                 43
damages                                 43
found                                   25
                                      ... 
GARDENIA 3 WICK MORRIS BOXED CANDLE      2
GREEN 3 PIECE POLKADOT CUTLERY SET       2
GREEN BITTY LIGHT CHAIN                  2
HANGING HEART ZINC T-LIGHT HOLDER        2
wet/rusty                                2
Name: StockCode, Length: 172, dtype: int64


In [11]:
# Verifica a existência de StockCodes iguais para diferentes Description
duplicates_stock_codes = sales.groupby('StockCode')['Description'].nunique()

# Filtra os casos onde há mais de um Description para um mesmo StockCode
duplicates_stock_codes = duplicates_stock_codes[duplicates_stock_codes > 1]

# Imprime os StockCodes com mais de um Description associado
print("StockCodes com mais de um Description associado:")
print(duplicates_stock_codes.sort_values(ascending=False))

StockCodes com mais de um Description associado:
StockCode
20713           8
23084           7
85175           6
21830           6
85172           5
               ..
22777           2
22784           2
22785           2
22792           2
gift_0001_20    2
Name: Description, Length: 650, dtype: int64


#### Há dois casos observados:
#### 1. Um mesmo código possui descrições diferentes
#### 2. Códigos diferentes possuem descrições iguais.

#### Para a análise feita neste dataset, será considerado que esses casos são decorrentes de cadastros errados, duplicados, registros incorretos ou alterações propositais nas descrições. 

In [12]:
# Quantidade de valores únicos da coluna Description
sales.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

#### Para simplificar a análise e melhorar a identificação dos produtos, cada registro único de *[Description]* será considerado como um produto diferente, já que a quantidade valores únicos de StockCode e Description são bem próximos.